In [77]:
import time
import pickle

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data
import torch.nn.functional as F
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [197]:
data_x = np.load("data/linear_features/linear/k1p8nor3.npy")
# data_x = np.load("data/linear_features/point/k2p10nor2n10.npy")
# data_x = data_x.reshape(data_x.shape[0],-1)

data_y = np.load("data/linear_features/data_y.npy")
print(data_x.shape, data_y.shape)

(2432, 4, 9) (2432,)


In [198]:
x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.20, shuffle=True, random_state=12)


In [9]:
class NN2Layers(torch.nn.Module):
    def __init__(self, ninp: int, nhid: int, ntoken: int, dropout: float=0.0):
        
        super(NN2Layers, self).__init__()
        self.drop = nn.Dropout(dropout)
        self.nn1 = nn.Linear(ninp, nhid)
        self.nn2 = nn.Linear(nhid, ntoken)

        self.init_weights()

        self.ninp = ninp
        self.nhid = nhid
        self.ntoken = ntoken

    def init_weights(self):
        initrange = 0.1
        self.nn1.bias.data.zero_()
        self.nn1.weight.data.uniform_(-initrange, initrange)
        self.nn2.bias.data.zero_()
        self.nn2.weight.data.uniform_(-initrange, initrange)

    def forward(self, x: torch.tensor):

        output = self.nn1(x)
        output = self.drop(output)
        output = self.nn2(output)

        return output.softmax(dim=1)
 
    def predict(self, x: torch.tensor):
        # """預測並輸出機率大的類別

        # Args:
        #     x (torch.tensor): 詞 tensor。如果batch_first=True，input shape為（批次，序列），否則（序列，批次）。

        # Returns:
        #     [torch.tensor]: shape 與 x 一樣，但是序列為類別序列。
        # """
        output = self.forward(x)
        _, output = torch.max(output, 1)

        return output

In [217]:
class RNNTagger(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, tagset_size):
        super(RNNTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.rnn = nn.RNN(input_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        output, hn = self.rnn(sentence)
        tag_space = self.hidden2tag(output)
        tag_scores = F.log_softmax(tag_space, dim=1)[:, :, -1]
        return tag_scores

In [240]:
ntoken = 2

ninp = x_train.shape[2]
nhid = 256

model = RNNTagger(ninp, nhid, ntoken)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=4e-3, weight_decay=1e-4)
# optimizer = optim.SGD(model.parameters(), lr=4e-1)

batch_size = 8
epochs = 50000

tensor_x = torch.tensor(x_train).to(torch.float)
tensor_y = torch.tensor(y_train).to(torch.long)

test_x = torch.tensor(x_test).to(torch.float)
test_y = torch.tensor(y_test).to(torch.long)

# dataset = Data.TensorDataset(tensor_x, tensor_y)

# # train_set, valid_set = Data.random_split(dataset, [int(len(dataset) * 0.8), len(dataset) - int(len(dataset) * 0.8)], generator=torch.Generator().manual_seed(42))

# loader = Data.DataLoader(
#     dataset = dataset,
#     batch_size = batch_size,
# )

In [237]:
class Trainer():
    def __init__(self, model, optimizer, loss_fn):
        self.model = model
        self.optimizer = optimizer
        self.loss_fn = loss_fn
    
    def train(self, x, y, valid_x, valid_y, epochs = 2, batch_size = 1, epoch_print = True, sampler = None):
        # Early stopping
        the_last_loss = 100
        patience = 2
        trigger_times = 0
        
        train_dataset = Data.TensorDataset(x, y)
        loader = Data.DataLoader(
            dataset = train_dataset,
            batch_size = batch_size,
            sampler = sampler,
        )
        
        start_time = time.time()
        step_size = len(loader)
        loss_history = []
        for epoch in range(epochs):
            self.model.train()
            epoch_loss = 0
            epoch_time = time.time()
            for step, (batch_x, batch_y) in enumerate(loader):
                step_time = time.time()
                
                self.optimizer.zero_grad()
                pred_y = self.model(batch_x)
                loss = self.loss_fn(pred_y, batch_y)
                loss.backward()
                self.optimizer.step()
                
                loss_history.append(loss.item())
                epoch_loss += loss.item()
                # print('Epoch: %i | Step: %i/%i | Loss: %.2f | time: %.2f s' % (epoch, step, step_size, loss, time.time() - step_time))
            
            
            # Early stopping
            the_current_loss = self.validation(valid_x, valid_y, batch_size=batch_size)
            # print('The current loss:', the_current_loss)
            
            if the_current_loss >= the_last_loss:
                trigger_times += 1
                print('trigger times:', trigger_times)
                if trigger_times >= patience:
                    print('Early stopping!\nStart to test process.')
                    break
            else:
                print('trigger times: 0')
                trigger_times = 0
            
            if epoch_print:
                print('Epoch: %i | Loss: %.2f | time: %.2f s' % (epoch, the_current_loss, time.time() - epoch_time))
            
            the_last_loss = the_current_loss
            
        print('All Time: %.2f s | Loss: %.2f' % (time.time() - start_time, the_current_loss))
    
    def validation(self, valid_x, valid_y, batch_size = 1):
        train_dataset = Data.TensorDataset(valid_x, valid_y)
        valid_loader = Data.DataLoader(
            dataset = train_dataset,
            batch_size = batch_size,
        )
        self.model.eval()
        loss_total = 0

        # Test validation data
        with torch.no_grad():
            for step, (batch_x, batch_y) in enumerate(valid_loader):

                outputs = model(batch_x)
                loss = self.loss_fn(outputs, batch_y)
                loss_total += loss.item()

        return loss_total / len(valid_loader)

    def test(self, x, y):
        y_pred = self.model.predict(x)
        
        one_hot_y = np.eye(self.model.ntoken)[y]
        one_hot_y_pred = np.eye(self.model.ntoken)[y_pred]
        token_acc_array = []
        for i in range(self.model.ntoken):
            y_token = torch.tensor(one_hot_y[:, i])
            y_pred_token = torch.tensor(one_hot_y_pred[:, i])
            
            tp = (y_token * y_pred_token).sum(dim=0).to(torch.float32)
            tn = ((1 - y_token) * (1 - y_pred_token)).sum(dim=0).to(torch.float32)
            fp = ((1 - y_token) * y_pred_token).sum(dim=0).to(torch.float32)
            fn = (y_token * (1 - y_pred_token)).sum(dim=0).to(torch.float32)
            precision = tp / (tp + fp)
            rec = tp / (tp + fn)
            f1 = 2 * rec * precision / (rec + precision)
            token_acc_array.append(f1)
        acc = (y_pred == y).float().sum() / len(y)
        token_acc_array = torch.tensor(token_acc_array)
        return acc, token_acc_array

In [223]:
weight = 1. / np.unique(y_train, return_counts=True)[1]
samples_weight = np.array([weight[t] for t in tensor_y])

samples_weight = torch.from_numpy(samples_weight)
samples_weigth = samples_weight.double()
sampler = WeightedRandomSampler(samples_weight, len(samples_weight))

In [241]:
trainer = Trainer(model, optimizer, loss_fn)
trainer.train(tensor_x, tensor_y, test_x, test_y, epochs, batch_size, epoch_print=True, sampler=sampler)
# trainer.test(test_x, test_y)

trigger times: 0
Epoch: 0 | Loss: 1.42 | time: 0.04 s
trigger times: 0
Epoch: 1 | Loss: 1.40 | time: 0.05 s
trigger times: 1
Epoch: 2 | Loss: 1.46 | time: 0.04 s
trigger times: 0
Epoch: 3 | Loss: 1.43 | time: 0.06 s
trigger times: 1
Epoch: 4 | Loss: 1.52 | time: 0.03 s
trigger times: 0
Epoch: 5 | Loss: 1.52 | time: 0.04 s
trigger times: 0
Epoch: 6 | Loss: 1.39 | time: 0.05 s
trigger times: 1
Epoch: 7 | Loss: 1.63 | time: 0.05 s
trigger times: 0
Epoch: 8 | Loss: 1.47 | time: 0.04 s
trigger times: 0
Epoch: 9 | Loss: 1.40 | time: 0.05 s
trigger times: 1
Epoch: 10 | Loss: 1.54 | time: 0.04 s
trigger times: 0
Epoch: 11 | Loss: 1.42 | time: 0.04 s
trigger times: 1
Epoch: 12 | Loss: 1.59 | time: 0.04 s
trigger times: 2
Early stopping!
Start to test process.
All Time: 36.35 s | Loss: 1.76


In [43]:
from torch.utils.data.sampler import WeightedRandomSampler
numDataPoints = 1000
data_dim = 5
bs = 100

# Create dummy data with class imbalance 9 to 1
data = torch.FloatTensor(numDataPoints, data_dim)
target = np.hstack((np.zeros(int(numDataPoints * 0.9), dtype=np.int32),
                    np.ones(int(numDataPoints * 0.1), dtype=np.int32)))

print ('target train 0/1: {}/{}'.format(
    len(np.where(target == 0)[0]), len(np.where(target == 1)[0])))

class_sample_count = np.array(
    [len(np.where(target == t)[0]) for t in np.unique(target)])
weight = 1. / class_sample_count
samples_weight = np.array([weight[t] for t in target])

print(class_sample_count)
samples_weight = torch.from_numpy(samples_weight)
samples_weigth = samples_weight.double()
sampler = WeightedRandomSampler(samples_weight, len(samples_weight))

target = torch.from_numpy(target).long()
train_dataset = torch.utils.data.TensorDataset(data, target)

train_loader = Data.DataLoader(
    train_dataset, batch_size=bs, num_workers=1, sampler=sampler)

for i, (data, target) in enumerate(train_loader):
    print ("batch index {}, 0/1: {}/{}".format(
        i,
        len(np.where(target.numpy() == 0)[0]),
        len(np.where(target.numpy() == 1)[0])))

target train 0/1: 900/100
[900 100]
batch index 0, 0/1: 55/45
batch index 1, 0/1: 54/46
batch index 2, 0/1: 52/48
batch index 3, 0/1: 60/40
batch index 4, 0/1: 46/54
batch index 5, 0/1: 44/56
batch index 6, 0/1: 43/57
batch index 7, 0/1: 54/46
batch index 8, 0/1: 57/43
batch index 9, 0/1: 52/48
